In [ ]:
# 訓練資料集Google 2012-2016的股價。1258筆
# 測試資料集Google 2017的股價。20筆
!wget https://github.com/shhuangmust/AI/raw/111-2/Google_Stock_Price_Train.csv
!wget https://github.com/shhuangmust/AI/raw/111-2/Google_Stock_Price_Test.csv

--2024-10-26 15:40:17--  https://github.com/shhuangmust/AI/raw/111-2/Google_Stock_Price_Train.csv
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shhuangmust/AI/111-2/Google_Stock_Price_Train.csv [following]
--2024-10-26 15:40:18--  https://raw.githubusercontent.com/shhuangmust/AI/111-2/Google_Stock_Price_Train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63488 (62K) [text/plain]
Saving to: ‘Google_Stock_Price_Train.csv’

Google_Stock_Price_ 100%[===================>]  62.00K  --.-KB/s    in 0.007s  

2024-10-26 15:40:18 (8.96 MB/s) - ‘Google_Stock_Price_Train.csv’ saved [63488/63488]

--

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# 讀取訓練資料集
dataset_train = pd.read_csv('Google_Stock_Price_Train.csv')
# 取出Open股票指數欄位值
training_set = dataset_train.iloc[:, 1:2].values
# Feature Scaling：把股票指數變成0-1之間的數字
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

X_train = []   #預測點的前 60 天的資料
y_train = []   #預測點

for i in range(60, 1258):  # 1258 是訓練集總數
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])

# 轉成numpy array的格式，以利輸入RNN
X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
print(X_train.shape)
print(X_train[0])

(1198, 60, 1)
[[0.08581368]
 [0.09701243]
 [0.09433366]
 [0.09156187]
 [0.07984225]
 [0.0643277 ]
 [0.0585423 ]
 [0.06568569]
 [0.06109085]
 [0.06639259]
 [0.0614257 ]
 [0.07474514]
 [0.02797827]
 [0.02379269]
 [0.02409033]
 [0.0159238 ]
 [0.01078949]
 [0.00967334]
 [0.01642607]
 [0.02100231]
 [0.02280676]
 [0.02273235]
 [0.02810849]
 [0.03212665]
 [0.0433812 ]
 [0.04475779]
 [0.04790163]
 [0.0440695 ]
 [0.04648783]
 [0.04745517]
 [0.04873875]
 [0.03936305]
 [0.04137213]
 [0.04034898]
 [0.04784582]
 [0.04325099]
 [0.04356723]
 [0.04286033]
 [0.04602277]
 [0.05398467]
 [0.05738894]
 [0.05714711]
 [0.05569611]
 [0.04421832]
 [0.04514845]
 [0.04605997]
 [0.04412531]
 [0.03675869]
 [0.04486941]
 [0.05065481]
 [0.05214302]
 [0.05612397]
 [0.05818885]
 [0.06540665]
 [0.06882953]
 [0.07243843]
 [0.07993526]
 [0.07846566]
 [0.08034452]
 [0.08497656]]


In [ ]:
# Import the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense,Dropout,LSTM

regressor = Sequential()
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# 進行訓練
regressor.fit(X_train, y_train, epochs = 65, batch_size = 512)

Epoch 1/65
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 44ms/step - loss: 0.3119
Epoch 2/65
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - loss: 0.0837
Epoch 3/65
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0575
Epoch 4/65
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0176
Epoch 5/65
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0328
Epoch 6/65
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0228
Epoch 7/65
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0125
Epoch 8/65
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0153
Epoch 9/65
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0090
Epoch 10/65
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0103
Epoch 11/65
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0087
Epoch 12/65
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0071
Epoch 13/65
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0071
Epoch 14/65
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0063
Epoch 15/65
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0065
Epoch 16/65
3/3 ━━━━━━━━━━━━━━━━━━

In [ ]:
# 讀取2017股票測試資料集
dataset_test = pd.read_csv('Google_Stock_Price_Test.csv')
# 抓出OPEN股票指數欄位值
real_stock_price = dataset_test.iloc[:, 1:2].values

#抓出2017年前60天的資料
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs) # Feature Scaling
X_test = []
for i in range(60, 80):  # timesteps一樣60； 80 = 先前的60天資料+2017年的20天資料
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# 預測股票指數
predicted_stock_price = regressor.predict(X_test)
# 把預測結果從0~1之間還原為原來的股票指數
predicted_stock_price = sc.inverse_transform(predicted_stock_price)  # to get the original scale


In [ ]:
# Visualising the results
import matplotlib.pyplot as plt  # for 畫圖用
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')  # 紅線表示真實股價
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')  # 藍線表示預測股價
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()